In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 2000)
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from sklearn.metrics.pairwise import cosine_similarity
import re

In [2]:
#from google.colab import drive
#drive.mount('/content/drive')

In [3]:
# todo:
# other library for languages detection ? qualifiers included if exists? 
# duplicated titles: red queen 1, The Secret Garden
# variable ngram
# clean the titles --> DONE
# try out with other languages
# recommend for all the books

In [4]:
#pfad = "drive/MyDrive/DataMiningSeminar/"
items = pd.read_csv("data/items.csv", delimiter='|')
transactions = pd.read_csv("data/transactions.csv", delimiter='|')
evaluation = pd.read_csv("data/evaluation.csv", delimiter='|')
print(items.shape)
print(transactions.shape)
print(evaluation.shape)

(78030, 6)
(365143, 5)
(1000, 1)


In [5]:
items_new = pd.read_csv('items_language.csv')
titles_eng = items_new.loc[items_new.language == "en"]

# Test on evaluation dataset

In [6]:
df_merged = pd.merge(evaluation, items_new, on = "itemID", how="left")

In [7]:
df_merged = df_merged[~df_merged.title.isna()]

In [8]:
df_merged.language.value_counts()

en    486
de    315
es     33
id     21
it     16
fr     14
nl     14
tl     12
ca     11
af     10
no      8
pt      7
da      6
cy      6
sv      6
hu      5
ro      5
et      4
lt      3
sw      3
pl      3
tr      3
so      2
sq      1
vi      1
fi      1
sk      1
hr      1
Name: language, dtype: int64

In [9]:
df_merged[df_merged.language == "en"]

,itemID,title,author,publisher,main topic,subtopics,language
0,12,Breathtaking,Reva Ryann Thompson,Xlibris,FM,[],en
1,45274,The Dead Man in Indian Creek,Mary Downing Hahn,AVON BOOKS,YFCF,"[YFZR,YXHB]",en
2,10104,The Humble Seed,Linda Appleby,Linda Appleby,YFB,[YX],en
4,14015,The Man of Steel: Superman vs. the Moon Bandits,Scott Sonneborn,STONE ARCH BOOKS,YFCF,[XQK],en
5,28515,Singing Innocence and Experience,Sonya Taaffe,Prime,FM,[],en
7,1822,School of Fear,Gitty Daneshvari,LITTLE BROWN BOOKS FOR YOUNG R,YFD,[YFH],en
9,60188,Evil Emperor Penguin: Winging It,Laura Ellen Anderson,David Fickling Books,XADC,[YF],en
11,31401,Ben's Rubbish Day,Naomi Webb,Olympia Publishers,YFC,[],en
14,73378,Charlie Bone and the Blue Boa,Jenny Nimmo,Egmont UK Ltd,YFH,[5AK],en
15,25049,The Hidden Star,K. Sello Duiker,Cassava Republic Press,YFH,[],en


# Prepocessing title

In [10]:
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

In [11]:
# lowercase
def lower_case(title):
    return np.char.lower(title)
#remove the stop words

def remove_stop_words(title, lang):
    stop_words = set(stopwords.words(lang))
    word_tokens = word_tokenize(title) 
  
    filtered_title = [] 
  
    for w in word_tokens: 
        if w not in stop_words: 
            filtered_title.append(w)
    return filtered_title
# remove punctuation:
def remove_punctuation(title):
    symbols = "!\"#$%&()*+-./:';<=>?@[\]^_`{|}~\n"
    filtered_title = [] 
  
    for t in title: 
        if t not in symbols: 
            filtered_title.append(str(t))
    return filtered_title
# remove single characters
#def remove_single_characters(text):
#    new_text = ""
#    for t in text:
#        if len(t) > 1:
#            new_text = new_text + " " + t
#    return new_text
#stemmer = PorterStemmer()

#def stemming_text(title):
    
#    filtered_title = []
#    for t in title:
#        filtered_title.append(stemmer.stem(t))
#    return filtered_title


In [ ]:
titles_eng["title_processed"] = titles_eng.title.apply(lambda x: lower_case(str(x)))
titles_eng["title_processed"] = titles_eng.title_processed.apply(lambda x: remove_stop_words(str(x), "english"))
titles_eng["title_processed"] = titles_eng.title_processed.apply(lambda x: remove_punctuation(x))
#titles_eng["title_processed"] = titles_eng.title_processed.apply(lambda x: remove_single_characters(str(x)))
#titles_eng["title_processed"] = titles_eng.title_processed.apply(lambda x: stemming_text(x))


/Users/thuylinhnguyen/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
eng = titles_eng.copy()
eng['author'] = eng['author'].map(lambda x: x.split(',') if str(x) != "nan" else "nan")
eng['main topic'] = eng['main topic'].map(lambda x: x.split(',') if str(x) != "nan" else "nan")

#df["author"] = df["author"].apply(lambda x: i.lower().replace(' ','') for i in x)
l = []
for index, row in eng.iterrows():
    l.append([x.lower().replace(' ','') for x in row['author']])
eng["author"] = l
#    row['main topic'] = [x.lower().replace(' ','') for x in row['main topic']]

In [ ]:
l = []
columns = ['main topic', 'author', 'title_processed']
for index, row in eng.iterrows():
    words = ''
    for col in columns:
        if str(row[col]) != "nan":
            words += ' '.join(row[col]) + ' '
    l.append(words)
eng["Bag_of_words"] = l    
eng = eng[['title','Bag_of_words']]

In [ ]:
eng.head()

In [ ]:
count = CountVectorizer()
count_matrix = count.fit_transform(eng['Bag_of_words'])
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [ ]:
#def cosine_similarity_matrix(df, lang):
#    #Converting the book title into vectors and used bigram
#    tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df = 1, stop_words=lang)
#    tfidf_matrix = tf.fit_transform(df['title'])
#    cosine_similarities = cosine_similarity(tfidf_matrix, tfidf_matrix) 

In [ ]:
#cosine_similarity_matrix(titles_eng, 'english')

In [ ]:
def recommendations(title, cosine_sim = cosine_sim):
    # initializing the empty list of recommended movies
    recommendations = [] 
    indices = pd.Series(eng['title'])
    
    # gettin the index of the book that matches the title
    idx = indices[indices == title].index[0]
    
    ##if title not in idx:
    ##    raise KeyError("title is not in indices")
    
    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indices of the 5 most similar books
    top_5_indices = list(score_series.iloc[1:6].index)

    # populating the list with the titles of the best 5 matching books
    for i in top_5_indices:
        recommendations.append(list(eng["title"])[i])

    return recommendations

In [ ]:
recommendations("The Railway Children")

In [ ]:
titles_eng.title.value_counts()

# Test for german books

In [ ]:
titles_german = items_new.loc[items_new.language == "de"]

In [ ]:
titles_german.head()

In [ ]:
titles_german["title_processed"] = titles_german.title.apply(lambda x: lower_case(str(x)))
titles_german["title_processed"] = titles_german.title_processed.apply(lambda x: remove_stop_words(str(x), "german"))
titles_german["title_processed"] = titles_german.title_processed.apply(lambda x: remove_punctuation(x))

In [ ]:
ger = titles_german.copy()
ger['author'] = ger['author'].map(lambda x: x.split(',') if str(x) != "nan" else "nan")
ger['main topic'] = ger['main topic'].map(lambda x: x.split(',') if str(x) != "nan" else "nan")

#df["author"] = df["author"].apply(lambda x: i.lower().replace(' ','') for i in x)
l = []
for index, row in ger.iterrows():
    l.append([x.lower().replace(' ','') for x in row['author']])
ger["author"] = l

In [ ]:
l = []
columns = ['main topic', 'author', 'title_processed']
for index, row in ger.iterrows():
    words = ''
    for col in columns:
        if str(row[col]) != "nan":
            words += ' '.join(row[col]) + ' '
    l.append(words)
ger["Bag_of_words"] = l    
ger = ger[['title','Bag_of_words']]

In [ ]:
count = CountVectorizer()
count_matrix = count.fit_transform(ger['Bag_of_words'])
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [ ]:
def recommendations(title, cosine_sim = cosine_sim):
    # initializing the empty list of recommended movies
    recommendations = [] 
    indices = pd.Series(ger['title'])
    
    # gettin the index of the book that matches the title
    idx = indices[indices == title].index[0]
    
    ##if title not in idx:
    ##    raise KeyError("title is not in indices")
    
    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indices of the 5 most similar books
    top_5_indices = list(score_series.iloc[1:6].index)

    # populating the list with the titles of the best 5 matching books
    for i in top_5_indices:
        recommendations.append(list(ger["title"])[i])

    return recommendations

In [ ]:
recommendations("Meine Kindergarten-Freunde (Pirat)")

In [ ]:
'''def recommend(title):# Convert the index into series
    indices = pd.Series(titles_eng.index, index = titles_eng['title'])
    
    #Converting the book title into vectors and used bigram
    tf = TfidfVectorizer(analyzer='word', ngram_range=(2, 2), min_df = 1, stop_words='english')
    tfidf_matrix = tf.fit_transform(titles_eng['title'])
    
    # Calculating the similarity measures based on Cosine Similarity
    sg = cosine_similarity(tfidf_matrix, tfidf_matrix)
    
    # Get the index corresponding to original_title
    idx = indices[title]
    # Get the pairwsie similarity scores 
    sig = list(enumerate(sg[idx]))
    # Sort the books
    sig = sorted(sig, key=lambda x: x[1], reverse=True)
    # Scores of the 5 most similar books 
    sig = sig[1:6]
    # Book indicies
    book_indices = [i[0] for i in sig]
   
    # Top 5 book recommendation
    rec = titles_eng[['title']].iloc[book_indices]
    print(rec)'''